# Reconstruction: quadratic 

The spatial reconstruction of the fundamental QQNM compared with the corresponding spatial predictions, plotted on the sphere in Mollweide projection. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import CCE
from qnmfits.spatial_mapping_functions import *

plt.style.use("stylesheet.mplstyle")
fig_width = 246.0 * (1.0 / 72.27)
fig_height = fig_width / 1.618

In [ ]:
id = "0001"
sim = CCE.SXS_CCE(id)

chif = sim.chif_mag
Mf = sim.Mf

l_max = 8
n_max = 7

t0 = 8

In [ ]:
mapping = [(2, 2, 0, 1, 2, 2, 0, 1)]
map = mapping[0]

QNMs = [
    (lam, 4, n, p)
    for lam in np.arange(4, l_max + 1)
    for n in np.arange(0, n_max + 1)
    for p in (-1, +1)
]

best_fit = mapping_multimode_ringdown_fit(
    sim.times,
    sim.h,
    modes=QNMs.copy(),
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
    mapping_modes=mapping,
    spherical_modes=[(lam, 4) for lam in np.arange(4, l_max + 1)],
)

In [ ]:
# Compute the spatial mismatch for the various predictions

smA = spatial_mismatch_quadratic(best_fit, map, l_max, chif, Qmu_A)
smBi = spatial_mismatch_quadratic(best_fit, map, l_max, chif, Qmu_B, s1=-2, s2=0)
smBii = spatial_mismatch_quadratic(best_fit, map, l_max, chif, Qmu_B, s1=-1, s2=-1)
smC = spatial_mismatch_quadratic(best_fit, map, l_max, chif, Qmu_C)
smD = spatial_mismatch_quadratic(best_fit, map, l_max, chif, Qmu_D)

print("A:", smA)
print("Bi:", smBi)
print("Bii:", smBii)
print("C:", smC)
print("D:", smD)

In [ ]:
# Compute the spatial reconstruction for the various predictions

lon = np.linspace(-np.pi, np.pi, 200)
lat = np.linspace(-np.pi / 2, np.pi / 2, 200)
Lon, Lat = np.meshgrid(lon, lat)

map = (2, 2, 0, 1, 2, 2, 0, 1)

F = spatial_reconstruction(np.pi / 2 - Lat, Lon, best_fit, map, l_max)
G_A = spatial_prediction_quadratic(
    np.pi / 2 - Lat, Lon, map, l_max, chif, Qmu_A, s1=-2, s2=-2, s3=-4
)
G_Bi = spatial_prediction_quadratic(
    np.pi / 2 - Lat, Lon, map, l_max, chif, Qmu_B, s1=-2, s2=0
)
G_Bii = spatial_prediction_quadratic(
    np.pi / 2 - Lat, Lon, map, l_max, chif, Qmu_B, s1=-1, s2=-1
)
G_C = spatial_prediction_C(np.pi / 2 - Lat, Lon, map, chif)
G_D = spatial_prediction_quadratic(np.pi / 2 - Lat, Lon, map, l_max, chif, Qmu_D)

In [ ]:
fig, ax = plt.subplots(
    figsize=(fig_width * 2, fig_height * 1.5),
    nrows=2,
    ncols=3,
    subplot_kw={"projection": "mollweide"},
)

ax[0, 0].title.set_text("Reconstruction")
ax[0, 0].pcolormesh(Lon, Lat, np.real(F), cmap=plt.cm.RdBu)
ax[0, 0].set_xticks(np.radians([-90, 0, 90]))
ax[0, 0].set_yticks(np.radians([-60, 0, 60]))

ax[0, 1].title.set_text("Prediction A")
im3 = ax[0, 1].pcolormesh(Lon, Lat, np.real(G_A), cmap=plt.cm.RdBu)
ax[0, 1].set_xticks(np.radians([-90, 0, 90]))
ax[0, 1].set_yticks(np.radians([-60, 0, 60]))

ax[0, 2].title.set_text("Prediction B (i)")
ax[0, 2].pcolormesh(Lon, Lat, np.real(G_Bi), cmap=plt.cm.RdBu)
ax[0, 2].set_xticks(np.radians([-90, 0, 90]))
ax[0, 2].set_yticks(np.radians([-60, 0, 60]))

ax[1, 0].title.set_text("Prediction B (ii)")
ax[1, 0].pcolormesh(Lon, Lat, np.real(G_Bii), cmap=plt.cm.RdBu)
ax[1, 0].set_xticks(np.radians([-90, 0, 90]))
ax[1, 0].set_yticks(np.radians([-60, 0, 60]))

ax[1, 1].title.set_text("Prediction C")
ax[1, 1].pcolormesh(Lon, Lat, np.real(G_C), cmap=plt.cm.RdBu)
ax[1, 1].set_xticks(np.radians([-90, 0, 90]))
ax[1, 1].set_yticks(np.radians([-60, 0, 60]))

ax[1, 2].title.set_text("Prediction D")
ax[1, 2].pcolormesh(Lon, Lat, np.real(G_D), cmap=plt.cm.RdBu)
ax[1, 2].set_xticks(np.radians([-90, 0, 90]))
ax[1, 2].set_yticks(np.radians([-60, 0, 60]))

cax = fig.add_axes([0.25, 0, 0.51, 0.04])
fig.colorbar(im3, cax=cax, orientation="horizontal")
fig.text(
    0.5,
    0.065,
    r"$\mathrm{Re}(A_{\Tilde{\alpha} \Tilde{\alpha} '}(\theta , \phi))$",
    ha="center",
)

fig.savefig(
    "figs/quadratic_fundamental_mapping_reconstruction.png",
    dpi=600,
    bbox_inches="tight",
)
plt.show()